# Atlas Processing - Initial Image Creation

- Documentation: https://nilearn.github.io/auto_examples/01_plotting/plot_surf_atlas.html
- Note: nilearn's 'surface.load_surface_data' function does not suppport .mgh / .mgz files; must use mri_convert or mris_convert (part of Freesurfer) to first convert to an acceptable format, e.g. .nii

In [1]:
import nilearn
from nilearn import surface
from nilearn import plotting

import os
import numpy as np
import matplotlib.pyplot as plt

# import pandas as pd
# import png  # for reloading / working with previously saved images

/home/groups/jyeatman/software/anaconda3/envs/torch-ni-ny/lib/python3.9/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


## Set up directories

In [2]:
# === DIRECTORIES === #

# input data directories
# overall format:
# -- hbn_dir/sub-{EID}/label_subdir/parc_filename

hbn_dir = '/scratch/users/samjohns/HBN/BIDS_curated/derivatives/freesurfer'
label_subdir = '/label'
surf_subdir = '/surf'

curv_filename = 'lh.curv'
infl_filename = 'lh.inflated'
pial_filename = 'lh.pial'
parc_filename = 'lh.aparc.a2009s.annot'

# output data directories
out_data_dir = '/scratch/groups/jyeatman/samjohns-projects/data'
image_out_subdir = 'parc-images'
image_out_dir = out_data_dir + '/' + image_out_subdir
image_out_dir
os.makedirs(image_out_dir, exist_ok=True)  # ensure image output directory exists
assert os.path.exists(image_out_dir)

# === LABELS === #

# important:
# select a subset of labels that are visible in  ventral view
# label = 43 was borderline and removed for convenience
labels_to_plot = [2, 19, 21, 23, 24, 25, 30, 37, 38, 50, 51, 57, 58, 59, 60, 61, 63, 65]

In [3]:
subjects = [s for s in os.listdir(hbn_dir) if 'sub-' in s]
subjects.sort()

In [4]:
len(subjects)

2569

## Make images

In [6]:
def make_angles(njitter=20, scale=30, base_angle=None):
    noise = scale * (np.random.rand(njitter, 2) - 0.5)
    if base_angle is None:
        base_angle = np.array([210.0, 90.0])
    return noise + base_angle

In [10]:
out_base_dir = '/scratch/groups/jyeatman/samjohns-projects/data'
out_parc_dir = out_base_dir + '/parc-images-jitter'
out_curv_dir = out_base_dir + '/curv-images-jitter'
os.makedirs(out_parc_dir)
os.makedirs(out_curv_dir)

In [22]:
def make_subject_images(sub, out_parc_path, out_curv_path, jitter=True, make_curv=True):
    
    parc_path = f'{hbn_dir}/{sub}{label_subdir}/{parc_filename}'
    curv_path = f'{hbn_dir}/{sub}{surf_subdir}/{curv_filename}'
    infl_path = f'{hbn_dir}/{sub}{surf_subdir}/{infl_filename}'
    
    if jitter:
        angles = make_angles()
    else:
        angles = [[210.0, 90.0]]
    
    # check files exist
    if not (os.path.exists(parc_path) 
            and os.path.exists(curv_path) 
            and os.path.exists(infl_path)
           ):
        return

    parc = surface.load_surf_data(parc_path)
    curv = surface.load_surf_data(curv_path)
    infl = surface.load_surf_mesh(infl_path)
    
    selected_parc = np.array([labels_to_plot.index(l) if l in labels_to_plot else -1 for l in parc])
    
    for a in angles:
        fig, ax = plt.subplots(figsize=(8, 8))
        plotting.plot_surf_roi(infl, selected_parc
                               ,view=(a[0], a[1])
                               # ,bg_map=test_curv
                               # ,bg_on_data=True 
                               ,figure=fig
                               ,cmap='tab20'
                               ,output_file=f'{out_parc_path}/{sub}-{a[0]:.2f}-{a[1]:.2f}-parc.png'   
                               # ,threshold=25.0
                               # colorbar=True
                              )

        if make_curv:
            fig, ax = plt.subplots(figsize=(8, 8))
            plotting.plot_surf_roi(infl, selected_parc
                                   ,view=(a[0], a[1])
                                   ,bg_map=curv
                                   # ,bg_on_data=True 
                                   ,figure=fig
                                   ,cmap='tab20'
                                   ,threshold=25.0
                                   ,output_file=f'{out_curv_path}/{sub}-{a[0]:.2f}-{a[1]:.2f}-curv.png'
                                   # colorbar=True
                                  )
        
    return

In [23]:
# main loop: don't execute this cell unless you want to go grab a coffee
for sub in subjects[:1]:
    make_subject_images(sub, out_parc_path=out_parc_dir, out_curv_path=out_curv_dir)

In [ ]:
os.makedirs()